<a href="https://colab.research.google.com/github/vijaynadimpalli/numer_ai_experiments/blob/master/Numerai_Trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [2]:
!git clone https://github.com/vijaynadimpalli/numer_ai_experiments.git

Cloning into 'numer_ai_experiments'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
from numer_ai_experiments.utils import *
from numer_ai_experiments.tf_classes import *

In [ ]:
# install dependencies
try:
  import numerapi
except:
  !pip install numerapi
  import numerapi
 
 
# import dependencies
import pandas as pd
import numpy as np
import gc,os
 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Dropout,BatchNormalization,Activation,LSTM,Lambda,Bidirectional,GaussianNoise,Concatenate
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy,LogCosh
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.activations import swish
 
from tensorflow.keras.mixed_precision import experimental as mixed_precision
 
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split,GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
 
import random
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz",nrows=1)
dtype_dict = {x:np.float32 for x in training_data.columns if ('feature' in x or 'target' in x)}
#Not using float16 because np.linalg in _neutralize function does not support it.

training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz",dtype=dtype_dict)


#Centering all features and target data around 0 (for good NN convergence)
training_data['target'] -= 0.5

#Using .loc and mask for below operation uses a lot of  memory 
for col in (x for x in training_data.columns if 'feature' in x):
  training_data[col] -= 0.5

#data is float64 now...

#training_data = training_data.astype(dtype_dict)
#converting training data to float32 seems to have decreased the correlation from 427 to 407 to 380...

training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,-0.50,0.00,-0.25,-0.50,0.0,-0.25,-0.25,-0.25,0.25,0.25,-0.25,-0.25,0.50,0.25,0.0,0.5,0.0,-0.50,0.00,0.00,-0.50,-0.5,-0.50,0.50,-0.25,-0.50,0.00,-0.25,0.25,0.00,0.50,0.25,0.25,0.0,0.00,0.25,0.0,...,0.25,0.25,0.25,0.00,0.50,0.50,0.00,0.25,0.0,-0.25,-0.25,0.25,0.00,0.50,0.0,0.25,0.25,-0.25,0.00,0.50,0.25,0.00,0.00,0.50,-0.25,0.0,0.00,0.00,0.25,0.50,0.50,0.50,0.25,0.00,0.25,0.00,0.50,0.00,0.25,0.00
1,n0014af834a96cdd,era1,train,-0.50,-0.50,-0.50,-0.25,0.0,-0.50,-0.50,-0.25,0.00,0.00,-0.50,0.00,-0.50,0.00,0.0,0.0,0.0,0.00,-0.25,-0.25,0.00,-0.5,0.50,0.00,0.00,0.00,0.25,0.00,0.00,0.25,-0.25,0.00,0.25,0.0,-0.25,0.25,0.0,...,-0.25,-0.25,-0.25,0.50,0.50,0.00,0.00,0.00,-0.5,-0.25,0.50,0.00,0.50,0.50,0.0,0.00,0.00,0.50,-0.25,0.25,0.50,-0.25,-0.25,0.50,0.00,0.0,0.00,0.25,0.25,0.25,0.50,0.50,-0.50,-0.50,0.25,-0.25,-0.50,-0.25,0.50,-0.25
2,n001c93979ac41d4,era1,train,-0.25,0.00,-0.25,-0.25,0.5,0.25,0.25,-0.25,-0.50,-0.25,0.00,0.50,0.00,0.25,0.0,0.0,0.5,0.00,0.00,0.00,-0.25,-0.5,-0.25,0.25,0.25,0.25,0.00,0.25,0.00,-0.25,0.00,0.25,-0.25,0.0,0.00,0.25,0.0,...,-0.25,0.50,0.50,0.50,0.00,0.50,0.50,0.50,0.0,0.50,-0.50,0.50,0.50,0.00,0.5,0.25,0.50,-0.50,0.00,0.25,-0.50,0.50,0.00,0.00,0.25,0.5,0.25,0.50,-0.25,0.00,-0.25,0.00,-0.50,-0.50,0.00,0.50,-0.50,-0.25,0.25,-0.25
3,n0034e4143f22a13,era1,train,0.50,-0.50,-0.50,0.00,0.0,-0.25,-0.25,0.25,-0.25,0.00,0.00,0.00,0.25,0.00,0.5,0.0,0.0,-0.50,0.50,-0.50,0.25,-0.5,0.00,0.00,0.00,0.00,-0.50,0.00,0.00,0.25,0.25,0.00,-0.25,0.0,0.00,0.00,0.0,...,0.50,0.50,0.25,0.25,0.50,0.25,0.25,0.25,0.5,0.25,0.50,0.25,0.50,0.25,0.5,-0.50,0.00,0.25,0.50,0.25,0.50,0.25,0.50,0.50,-0.50,0.0,0.25,0.25,0.50,0.25,0.50,0.50,0.25,0.25,0.50,0.50,0.25,0.50,0.50,-0.25
4,n00679d1a636062f,era1,train,-0.25,-0.25,-0.25,-0.25,-0.5,-0.25,0.00,-0.25,-0.25,0.00,-0.25,-0.25,0.25,0.00,-0.5,0.0,0.0,-0.25,-0.50,0.00,-0.50,0.0,-0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,-0.25,0.0,0.00,0.00,0.0,...,0.50,-0.25,0.25,0.50,0.25,-0.50,-0.50,0.25,0.0,0.50,0.00,0.25,-0.25,0.00,-0.5,0.00,0.00,0.00,0.25,0.25,0.00,0.25,-0.25,0.25,0.00,0.0,-0.25,-0.25,0.25,0.00,0.25,0.25,-0.25,0.00,0.25,-0.50,0.00,-0.25,0.25,0.25


In [ ]:
gc.collect()

3624

In [ ]:
df = training_data
#120 eras
#Values of all features and targets are either 0,0.25,0.5,0.75,1

In [ ]:
# train_features = df[[x for x in df.columns if 'feature' in x]]
# train_targets = df[['target']]

# valid_features = tournament_data.query("data_type == 'validation'")[[x for x in df.columns if 'feature' in x]]
# valid_targets = tournament_data.query("data_type == 'validation'")[['target']]
#features.columns

X = df[[x for x in df.columns if 'feature' in x]].values
y = df[['target']].values

features = [x for x in df.columns if 'feature' in x]

# Tensorflow autoencoder

In [ ]:
def create_autoencoder(input_dim,output_dim,noise=0.09):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(128,activation='relu')(encoded)
 
    decoded = Dropout(0.2)(encoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
 
    x = Dense(128,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32,activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='linear',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=encoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])        
    return autoencoder, encoder
 
 
 
def create_model(input_shape,output_shape,params,encoder):
    inp = Input(input_shape)
    
    X = encoder(inp)
    X = Concatenate()([X,inp])
    X = BatchNormalization()(X)
    X = Dropout(params['dropout'][0])(X)
    
    for layer in range(len(params['units'])):
        X = Dense(params['units'][layer])(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
        X = Dropout(params['dropout'][layer + 1])(X)
        
    out = Dense(output_shape,activation = 'linear')(X)
    
    return Model(inputs=[inp],outputs=[out])
 
 
def run_model(X_train, X_test, y_train, y_test,params,encoder,callbacks=[]): 
 
    model = create_model(X_train.shape[-1],y_train.shape[-1],params,encoder)
    #print(model.summary())    
    
    model.compile(optimizer=Adam(learning_rate = params['lr']),loss=LogCosh(),metrics=[tf_correlation],
                  #run_eagerly=True
                  )
    
    model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=callbacks,epochs=params['epochs'],batch_size=params['batch_size'],verbose=params['verbose'])
    
    return model

In [ ]:
autoencoder, encoder = create_autoencoder(X.shape[-1],y.shape[-1],noise=0.1)
set_all_seeds(42)


autoencoder.compile(optimizer=Adam(0.001),loss={'decoded':'mse','label_output':'mse'})
print(autoencoder.summary())  
autoencoder.fit(X,(X,y),
                epochs=1000,
                batch_size=4096, 
                validation_split=0.1,
                callbacks=[EarlyStopping('val_loss',patience=30,restore_best_weights=True)],verbose=0)
encoder.save_weights('./encoder.hdf5')
encoder.trainable = False

K.clear_session()
gc.collect()

NameError: ignored

In [ ]:
gc.collect()
K.clear_session()

In [ ]:
#Code for baseline model without using hyperopt

# #1 iteration 830 seconds on tesla T4
# #final_score 0.022 for initial lr 1e-3
# #final_score 0.0123 for intial lr 1e-4

# best_hopt={
#     'dropout_0': 0.13133861210545011,
#  'dropout_1': 0.27060268015898126,
#  'dropout_2': 0.4889096286518324,
#  'dropout_3': 0.29340634238215346,
#  'dropout_4': 0.49916195991607193,
#  'units_1': 896,
#  'units_2': 512,
#  'units_3': 512,
#  'units_4': 896}


# para = best_hopt
# print(para)
# params = {}
# params['batch_size'] = 4096
# params['lr'] = 1e-3
# params['units'] = [para[x] for x in list(para.keys()) if 'units' in x]
# params['dropout'] = [para[x] for x in list(para.keys()) if 'dropout' in x]
# params['epochs'] = 100
# params['label_smoothing'] = 1e-2
# params['verbose'] = 2


# set_all_seeds(42)

# gkfold = GroupKFold(5)


# scores = np.zeros(5)
# feature_expo_folds = []

# for iter,(train_index,test_index) in enumerate(gkfold.split(X,y,training_data['era'])):
#     X_train, y_train = X[train_index], y[train_index]
#     X_test, y_test = X[test_index], y[test_index]

#     df2 = df.iloc[test_index,:].copy()    

#     # imp = SimpleImputer(strategy='mean',copy=True,verbose=1)
#     # X_train = imp.fit_transform(X_train)
#     # X_test = imp.transform(X_test)
#     ck_path = f"model_{iter}.h5"
    
#     ckpr = ModelCheckpoint(ck_path,monitor='val_tf_correlation',save_best_only=True,save_weights_only=True,mode='max',verbose=1)
#     early = EarlyStopping(monitor='val_tf_correlation',min_delta=1e-5,mode='max',patience=10,restore_best_weights=False,verbose=1)
#     rlr = ReduceLROnPlateau(monitor = 'val_tf_correlation', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-5, mode = 'max') 
#     #Bug in restore_best_weights https://github.com/keras-team/keras/issues/12511 
#     model = run_model(X_train, X_test, y_train, y_test,params,encoder,callbacks=[ckpr,early,rlr])
#     model.load_weights(ck_path)
#     y_pred = model.predict(X_test)


#     df2["preds"] = y_pred
#     df2["preds_neutralized"] = df2.groupby("era").apply(
#     lambda x: normalize_and_neutralize(x, ["preds"], features, 0.5) # neutralize by 50% within each era
#     )
#     scaler = MinMaxScaler()
#     df2["preds_neutralized"] = scaler.fit_transform(df2[["preds_neutralized"]]) # transform back to 0-1

#     # unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds"])[0,1])
#     # balanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds_neutralized"])[0,1])

#     spearman_unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds"]))
#     spearman_balanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds_neutralized"]))

#     # print(f"score for high feature exposure: {unbalanced_scores_per_era.mean()}")
#     # print(f"score for balanced feature expo: {balanced_scores_per_era.mean()}")
#     print(f"Spearman score for high feature exposure: {spearman_unbalanced_scores_per_era.mean()}")
#     print(f"Spearman score for balanced feature expo: {spearman_balanced_scores_per_era.mean()}")

#     print(f"std for high feature exposure: {spearman_unbalanced_scores_per_era.std(ddof=0)}")
#     print(f"std for balanced feature expo: {spearman_balanced_scores_per_era.std(ddof=0)}")

#     try:
#       print(f"smart sharpe for high feature exposure: {smart_sharpe(spearman_unbalanced_scores_per_era)}")
#       print(f"smart sharpe for balanced feature expo: {smart_sharpe(spearman_balanced_scores_per_era)}")
#     except ZeroDivisionError:
#       print("Division by zero!!!")
#       pass

#     try:
#       print(f"smart sortino for high feature exposure: {smart_sortino_ratio(spearman_unbalanced_scores_per_era)}")
#       print(f"smart sortino for balanced feature expo: {smart_sortino_ratio(spearman_balanced_scores_per_era)}")
#     except ZeroDivisionError:
#       print("Division by zero!!!")
#       pass





#     #Calculating feature exposures...
#     # feat_expo = []
#     # for feature in features:
#     #     feat_expo.append(np.corrcoef(df2[feature], df2["preds_neutralized"])[0,1])
#     # feat_expo = pd.Series(feat_expo, index=features)

#     # feature_expo_folds.append(feat_expo)

    

#     scores[iter] = spearman_unbalanced_scores_per_era.mean()

#     #Transfer Learning...
#     # p = params.copy()
#     # p['lr'] = p['lr']/100
#     # p['epochs'] = 3
#     # model = create_model(X_test.shape[-1],y_test.shape[-1],p,encoder)
#     # model.compile(optimizer=Adam(learning_rate = p['lr']),loss=BinaryCrossentropy(label_smoothing=p['label_smoothing']),metrics=[AUC(name='AUC')])
#     # model.load_weights(ck_path)
#     # model.fit(X_test,y_test,epochs=p['epochs'],batch_size=p['batch_size'],verbose=p['verbose'])
#     # model.save_weights(ck_path)

#     K.clear_session()
#     del model,X_train, y_train,X_test, y_test,y_pred, df2
#     rubbish = gc.collect()
#     #print('Memory used: ' + str(psutil.virtual_memory().used // 1e6))
    
    
# for iter,score in enumerate(scores):
#     print(f"Score for fold {iter+1} is : {score}")

# final_score = scores.mean()
# print(f"Final Score is : {final_score}")

In [ ]:
#feature_expo_folds

In [ ]:
from hyperopt import hp, fmin, tpe, Trials,space_eval
from hyperopt.pyll.base import scope

gc.collect()
K.clear_session()

In [ ]:
# def optimise(para):
#   print(para)
#   params = {}
#   params['batch_size'] = 4096
#   params['lr'] = 1e-3
#   params['units'] = para['units'][1:]
#   params['dropout'] = para['dropout']
#   params['epochs'] = 100
#   params['label_smoothing'] = 1e-2
#   params['verbose'] = 0


#   set_all_seeds(42)
#   gkfold = GroupKFold(5)
#   scores = np.zeros(5)
#   feature_expo_folds = []

#   for iter,(train_index,test_index) in enumerate(gkfold.split(X,y,training_data['era'])):
#     X_train, y_train = X[train_index], y[train_index]
#     X_test, y_test = X[test_index], y[test_index]

#     df2 = df.iloc[test_index,:].copy()    

#     # imp = SimpleImputer(strategy='mean',copy=True,verbose=1)
#     # X_train = imp.fit_transform(X_train)
#     # X_test = imp.transform(X_test)
#     ck_path = f"model_{iter}.h5"
    
#     ckpr = ModelCheckpoint(ck_path,monitor='val_tf_correlation',save_best_only=True,save_weights_only=True,mode='max',verbose=1)
#     early = EarlyStopping(monitor='val_tf_correlation',min_delta=1e-5,mode='max',patience=10,restore_best_weights=False,verbose=1)
#     rlr = ReduceLROnPlateau(monitor = 'val_tf_correlation', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-5, mode = 'max') 
#     #Bug in restore_best_weights https://github.com/keras-team/keras/issues/12511 
#     model = run_model(X_train, X_test, y_train, y_test,params,encoder,callbacks=[ckpr,early,rlr])
#     model.load_weights(ck_path)
#     y_pred = model.predict(X_test)


#     df2["preds"] = y_pred
#     df2["preds_neutralized"] = df2.groupby("era").apply(
#     lambda x: normalize_and_neutralize(x, ["preds"], features, 0.5) # neutralize by 50% within each era
#     )
#     scaler = MinMaxScaler()
#     df2["preds_neutralized"] = scaler.fit_transform(df2[["preds_neutralized"]]) # transform back to 0-1

#     # unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds"])[0,1])
#     # balanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds_neutralized"])[0,1])

#     spearman_unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds"]))
#     spearman_balanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds_neutralized"]))

#     # print(f"score for high feature exposure: {unbalanced_scores_per_era.mean()}")
#     # print(f"score for balanced feature expo: {balanced_scores_per_era.mean()}")
#     print(f"Spearman score for high feature exposure: {spearman_unbalanced_scores_per_era.mean()}")
#     print(f"Spearman score for balanced feature expo: {spearman_balanced_scores_per_era.mean()}")

#     print(f"std for high feature exposure: {spearman_unbalanced_scores_per_era.std(ddof=0)}")
#     print(f"std for balanced feature expo: {spearman_balanced_scores_per_era.std(ddof=0)}")

#     try:
#       print(f"smart sharpe for high feature exposure: {smart_sharpe(spearman_unbalanced_scores_per_era)}")
#       print(f"smart sharpe for balanced feature expo: {smart_sharpe(spearman_balanced_scores_per_era)}")
#     except ZeroDivisionError:
#       print("Division by zero!!!")
#       pass

#     try:
#       print(f"smart sortino for high feature exposure: {smart_sortino_ratio(spearman_unbalanced_scores_per_era)}")
#       print(f"smart sortino for balanced feature expo: {smart_sortino_ratio(spearman_balanced_scores_per_era)}")
#     except ZeroDivisionError:
#       print("Division by zero!!!")
#       pass


#     #Calculating feature exposures...
#     # feat_expo = []
#     # for feature in features:
#     #     feat_expo.append(np.corrcoef(df2[feature], df2["preds_neutralized"])[0,1])
#     # feat_expo = pd.Series(feat_expo, index=features)

#     # feature_expo_folds.append(feat_expo)    

#     scores[iter] = smart_sharpe(spearman_balanced_scores_per_era)

#     K.clear_session()
#     del model,X_train, y_train,X_test, y_test,y_pred, df2
#     rubbish = gc.collect()
#     #print('Memory used: ' + str(psutil.virtual_memory().used // 1e6))
      
      
#   for iter,score in enumerate(scores):
#     print(f"Score for fold {iter+1} is : {score}")

#   final_score = scores.mean()
#   print(f"Final Score is : {final_score}")
#   return -final_score    
    


# param_space = {}

# num_layers = 5
# param_space['units'] = [None]*num_layers
# param_space['dropout'] = [None]*num_layers
# param_space['dropout'][0] = hp.uniform('dropout_0',0.1,0.5)

# for ind in np.arange(1,num_layers):
#   param_space['units'][ind] = hp.choice(f'units_{ind}',np.arange(128,1024,128))
#   param_space['dropout'][ind] = hp.uniform(f'dropout_{ind}',0.1,0.5)

# trials = Trials()

# hopt = fmin(fn = optimise, 
#             space = param_space, 
#             algo = tpe.suggest, 
#             max_evals = 50, 
#             trials = trials, 
#            )

In [ ]:
# best_hopt = space_eval(param_space, hopt)
# best_hopt








#HP V1
#validation scores...
# Spearman score for high feature exposure: 0.02024075585765478
# Spearman score for balanced feature expo: 0.020506771934733794
# std for high feature exposure: 0.03409353952777108
# std for balanced feature expo: 0.030208522173948052
# smart sharpe for high feature exposure: nan
# smart sharpe for balanced feature expo: nan
"""

scores[iter] = spearman_unbalanced_scores_per_era.mean()


best loss: -0.04273940296590138

{'dropout': (0.10847936278162261,
  0.19700177820801823,
  0.289236771380973,
  0.41188042009646997,
  0.1978840381104568),
 'units': (None, 512, 768, 384, 896)}
 """

#HP V2
#validation scores...
# Spearman score for high feature exposure: 0.021036872796273743
# Spearman score for balanced feature expo: 0.021122382375489433
# std for high feature exposure: 0.028269333087386668
# std for balanced feature expo: 0.02364757360596254
# smart sharpe for high feature exposure: nan
# smart sharpe for balanced feature expo: nan
"""

  scores[iter] = smart_sharpe(spearman_balanced_scores_per_era)
  #Taking the smart sharpe ratio on neutralised data as score for hyperopt to optimize

  best loss: -1.434694787023735

    {'dropout': (0.18390444344120305,
  0.3011358381508519,
  0.10337490449824736,
  0.1021643294765179,
  0.16387070003988394),
 'units': (None, 896, 896, 512, 896)}

 """


"\n\n  scores[iter] = smart_sharpe(spearman_balanced_scores_per_era)\n  #Taking the smart sharpe ratio on neutralised data as score for hyperopt to optimize\n\n  best loss: -1.434694787023735\n\n    {'dropout': (0.18390444344120305,\n  0.3011358381508519,\n  0.10337490449824736,\n  0.1021643294765179,\n  0.16387070003988394),\n 'units': (None, 896, 896, 512, 896)}\n\n "

In [ ]:
#Using HP V2
best_hopt= {'dropout': (0.18390444344120305,
  0.3011358381508519,
  0.10337490449824736,
  0.1021643294765179,
  0.16387070003988394),
 'units': (None, 896, 896, 512, 896)}
 
 
para = best_hopt
print(para)
params = {}
params['batch_size'] = 4096
params['lr'] = 1e-3
params['units'] = para['units'][1:]
params['dropout'] = para['dropout']
params['epochs'] = 100
params['label_smoothing'] = 1e-2
params['verbose'] = 2

{'dropout': (0.18390444344120305, 0.3011358381508519, 0.10337490449824736, 0.1021643294765179, 0.16387070003988394), 'units': (None, 896, 896, 512, 896)}


In [ ]:
set_all_seeds(42)

gkfold = GroupKFold(5)


scores = np.zeros(5)
feature_expo_folds = []

for iter,(train_index,test_index) in enumerate(gkfold.split(X,y,training_data['era'])):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    df2 = df.iloc[test_index,:].copy()    

    # imp = SimpleImputer(strategy='mean',copy=True,verbose=1)
    # X_train = imp.fit_transform(X_train)
    # X_test = imp.transform(X_test)
    ck_path = f"model_{iter}.h5"
    
    ckpr = ModelCheckpoint(ck_path,monitor='val_tf_correlation',save_best_only=True,save_weights_only=True,mode='max',verbose=1)
    early = EarlyStopping(monitor='val_tf_correlation',min_delta=1e-5,mode='max',patience=10,restore_best_weights=False,verbose=1)
    rlr = ReduceLROnPlateau(monitor = 'val_tf_correlation', factor = 0.1, patience = 4, verbose = 1, min_delta = 1e-5, mode = 'max') 
    #Bug in restore_best_weights https://github.com/keras-team/keras/issues/12511 
    model = run_model(X_train, X_test, y_train, y_test,params,encoder,callbacks=[ckpr,early,rlr])
    model.load_weights(ck_path)
    y_pred = model.predict(X_test)


    df2["preds"] = y_pred
    df2["preds_neutralized"] = df2.groupby("era").apply(
    lambda x: normalize_and_neutralize(x, ["preds"], features, 0.5) # neutralize by 50% within each era
    )
    scaler = MinMaxScaler()
    df2["preds_neutralized"] = scaler.fit_transform(df2[["preds_neutralized"]]) # transform back to 0-1

    # unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds"])[0,1])
    # balanced_scores_per_era = df2.groupby("era").apply(lambda d: np.corrcoef(d["target"], d["preds_neutralized"])[0,1])

    spearman_unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds"]))
    spearman_balanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds_neutralized"]))

    # print(f"score for high feature exposure: {unbalanced_scores_per_era.mean()}")
    # print(f"score for balanced feature expo: {balanced_scores_per_era.mean()}")
    print(f"Spearman score for high feature exposure: {spearman_unbalanced_scores_per_era.mean()}")
    print(f"Spearman score for balanced feature expo: {spearman_balanced_scores_per_era.mean()}")

    print(f"std for high feature exposure: {spearman_unbalanced_scores_per_era.std(ddof=0)}")
    print(f"std for balanced feature expo: {spearman_balanced_scores_per_era.std(ddof=0)}")

    try:
      print(f"smart sharpe for high feature exposure: {smart_sharpe(spearman_unbalanced_scores_per_era)}")
      print(f"smart sharpe for balanced feature expo: {smart_sharpe(spearman_balanced_scores_per_era)}")
    except ZeroDivisionError:
      print("Division by zero!!!")
      pass

    try:
      print(f"smart sortino for high feature exposure: {smart_sortino_ratio(spearman_unbalanced_scores_per_era)}")
      print(f"smart sortino for balanced feature expo: {smart_sortino_ratio(spearman_balanced_scores_per_era)}")
    except ZeroDivisionError:
      print("Division by zero!!!")
      pass


    #Calculating feature exposures...
    # feat_expo = []
    # for feature in features:
    #     feat_expo.append(np.corrcoef(df2[feature], df2["preds_neutralized"])[0,1])
    # feat_expo = pd.Series(feat_expo, index=features)

    # feature_expo_folds.append(feat_expo)

    

    scores[iter] = spearman_unbalanced_scores_per_era.mean()

    #Transfer Learning...
    # p = params.copy()
    # p['lr'] = p['lr']/100
    # p['epochs'] = 3
    # model = create_model(X_test.shape[-1],y_test.shape[-1],p,encoder)
    # model.compile(optimizer=Adam(learning_rate = p['lr']),loss=BinaryCrossentropy(label_smoothing=p['label_smoothing']),metrics=[AUC(name='AUC')])
    # model.load_weights(ck_path)
    # model.fit(X_test,y_test,epochs=p['epochs'],batch_size=p['batch_size'],verbose=p['verbose'])
    # model.save_weights(ck_path)

    K.clear_session()
    del model,X_train, y_train,X_test, y_test,y_pred, df2
    rubbish = gc.collect()
    #print('Memory used: ' + str(psutil.virtual_memory().used // 1e6))
    
    
for iter,score in enumerate(scores):
    print(f"Score for fold {iter+1} is : {score}")

final_score = scores.mean()
print(f"Final Score is : {final_score}")

Epoch 1/100
98/98 - 5s - loss: 0.1237 - tf_correlation: -3.4254e-04 - val_loss: 0.0270 - val_tf_correlation: -5.2503e-03

Epoch 00001: val_tf_correlation improved from -inf to -0.00525, saving model to model_0.h5
Epoch 2/100
98/98 - 3s - loss: 0.0435 - tf_correlation: 0.0018 - val_loss: 0.0249 - val_tf_correlation: 0.0085

Epoch 00002: val_tf_correlation improved from -0.00525 to 0.00850, saving model to model_0.h5
Epoch 3/100
98/98 - 3s - loss: 0.0305 - tf_correlation: 6.6012e-04 - val_loss: 0.0245 - val_tf_correlation: 0.0143

Epoch 00003: val_tf_correlation improved from 0.00850 to 0.01432, saving model to model_0.h5
Epoch 4/100
98/98 - 3s - loss: 0.0267 - tf_correlation: 0.0043 - val_loss: 0.0243 - val_tf_correlation: 0.0199

Epoch 00004: val_tf_correlation improved from 0.01432 to 0.01990, saving model to model_0.h5
Epoch 5/100
98/98 - 3s - loss: 0.0255 - tf_correlation: 0.0066 - val_loss: 0.0246 - val_tf_correlation: 0.0192

Epoch 00005: val_tf_correlation did not improve from 0.

In [ ]:
# function ClickConnect(){

# console.log("Working"); 
# document.querySelector("#comments > span").click() 
# }
# setInterval(ClickConnect,5000)

## 4. Generate your first predictions
Now that we have a trained model, we can use it to make predictions on the tournament data.



In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
try:
  del training_data,X,y
  del tournament_data
except NameError:
  pass
gc.collect()

132

In [ ]:
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz",nrows=1)
dtype_dict = {x:np.float32 for x in tournament_data.columns if ('feature' in x or 'target' in x)}
#Not using float16 because np.linalg in _neutralize function does not support it.

tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
#tournament data 'validation':121 to 135 eras, 'test':others,'live':eraX

#Centering all features and target data around 0 (for good NN convergence)
#tournament_data['target'] -= 0.5

#Using .loc and mask for below operation uses a lot of  memory 
for col in (x for x in tournament_data.columns if 'feature' in x):
  tournament_data[col] -= 0.5

tournament_data = tournament_data.astype(dtype_dict)

tournament_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,-0.25,0.25,0.00,0.00,-0.5,0.25,0.0,-0.25,0.00,0.0,-0.25,-0.5,-0.25,0.00,-0.25,-0.50,-0.25,0.50,0.50,-0.25,0.50,0.50,-0.25,-0.25,-0.5,0.00,-0.25,0.25,-0.50,0.00,-0.25,-0.25,-0.25,0.00,-0.50,0.00,0.50,...,-0.50,-0.5,-0.25,0.0,-0.25,-0.25,-0.50,-0.25,-0.50,-0.25,0.00,0.00,0.00,0.00,-0.50,-0.25,0.25,-0.25,-0.25,0.00,-0.25,-0.50,-0.25,0.00,-0.25,0.0,-0.25,-0.25,0.50,0.25,0.25,0.25,0.50,0.25,0.00,0.0,0.5,-0.50,-0.50,0.25
1,n000920ed083903f,era121,validation,0.25,0.00,0.25,0.50,0.0,-0.50,-0.5,0.25,-0.25,-0.5,0.25,0.0,-0.50,-0.25,0.00,-0.50,0.50,-0.25,-0.25,0.50,0.50,-0.25,0.25,-0.50,-0.5,0.25,0.50,0.50,-0.50,-0.25,-0.50,-0.50,-0.25,-0.25,-0.25,-0.50,0.50,...,0.00,0.0,-0.25,0.5,0.00,-0.25,-0.50,-0.25,0.00,-0.25,0.50,-0.25,-0.50,0.00,0.25,0.25,0.00,0.50,0.50,-0.25,0.00,-0.25,0.00,0.00,0.00,0.0,-0.25,-0.25,0.25,0.00,0.00,0.00,0.25,0.50,0.25,0.0,0.0,0.00,0.00,0.50
2,n0038e640522c4a6,era121,validation,0.50,-0.50,-0.50,0.50,0.5,0.50,0.5,0.50,0.00,0.0,0.50,0.5,0.50,0.25,0.00,0.00,0.50,0.50,0.00,0.00,-0.50,0.50,0.00,0.50,0.0,0.50,0.00,0.50,-0.25,0.50,0.50,0.50,0.00,0.50,0.50,0.25,0.50,...,-0.25,0.0,-0.50,-0.5,-0.50,-0.25,-0.25,-0.50,0.00,-0.50,-0.50,-0.50,-0.25,-0.50,-0.25,0.00,-0.50,-0.50,-0.50,-0.50,-0.50,-0.50,0.00,-0.50,0.25,-0.5,-0.50,-0.25,-0.50,-0.50,-0.50,-0.50,0.00,-0.25,-0.50,-0.5,0.0,0.00,-0.50,1.00
3,n004ac94a87dc54b,era121,validation,0.25,0.50,0.50,0.00,-0.5,-0.50,-0.5,0.00,0.25,0.5,0.25,-0.5,0.00,-0.50,0.00,0.25,0.00,0.25,-0.25,0.25,-0.25,0.25,-0.25,0.25,0.5,0.00,0.00,0.25,0.00,0.50,0.00,-0.25,0.25,-0.25,0.25,-0.25,0.25,...,-0.50,-0.5,-0.50,-0.5,-0.50,-0.50,-0.50,-0.50,-0.25,-0.50,-0.25,-0.50,-0.50,-0.25,-0.50,-0.50,-0.50,-0.50,0.25,-0.50,-0.50,-0.25,-0.25,-0.50,-0.50,-0.5,-0.50,-0.50,-0.50,-0.50,-0.50,-0.50,-0.50,-0.25,-0.50,-0.5,-0.5,-0.25,-0.25,0.50
4,n0052fe97ea0c05f,era121,validation,-0.25,0.00,0.00,-0.25,0.5,0.00,0.0,-0.25,-0.25,0.0,0.00,0.5,0.50,0.50,0.50,0.25,0.00,0.00,0.00,0.25,-0.50,-0.50,-0.50,-0.25,-0.5,-0.50,0.25,-0.25,0.50,-0.25,0.50,0.25,-0.50,0.50,0.25,0.25,0.25,...,-0.50,0.0,0.00,-0.5,0.25,0.00,0.25,-0.25,-0.25,-0.25,-0.50,-0.25,0.00,-0.25,0.50,0.50,0.50,-0.50,-0.25,-0.50,-0.50,-0.25,-0.25,0.25,0.50,0.5,0.25,0.25,0.00,0.00,0.00,0.25,-0.50,-0.50,0.25,0.5,-0.5,-0.25,0.50,0.75


In [ ]:
# select the feature columns from the tournament data
features = [x for x in tournament_data.columns if 'feature' in x]
live_features = tournament_data[features]

In [ ]:
# predict the target on the live features
num_models = 5
predictions = np.zeros((len(tournament_data),1))
X_shape = 310 #X.shape[-1]
y_shape = 1 #y.shape[-1]


_, encoder = create_autoencoder(X_shape,y_shape,noise=0.1)
encoder.load_weights(f'encoder.hdf5')
encoder.trainable = False

for file_name in list(filter(lambda x: 'model' in x,os.listdir('.')))[:num_models]:        
    
    clf = create_model(X_shape,y_shape,params,encoder)
    clf.compile(optimizer=Adam(learning_rate = params['lr']),loss=LogCosh(),metrics=[tf_correlation],
                  #run_eagerly=True
                  )
    clf.load_weights(os.path.join('./',file_name))
    
    predictions += clf.predict(live_features)
    del clf
    gc.collect()
    print(f"{file_name} :: DONE")

predictions /= num_models

predictions += 0.5

NameError: ignored

In [ ]:
gc.collect()

In [ ]:
df2 = tournament_data

df2["preds"] = predictions
df2["preds_neutralized"] = df2.groupby("era").apply(
lambda x: normalize_and_neutralize(x, ["preds"], features, 0.5) # neutralize by 50% within each era
)
scaler = MinMaxScaler()
df2["preds_neutralized"] = scaler.fit_transform(df2[["preds_neutralized"]]) # transform back to 0-1

#df2 = tournament_data[tournament_data['data_type'] == 'validation']

spearman_unbalanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds"]))
spearman_balanced_scores_per_era = df2.groupby("era").apply(lambda d: correlation(d["target"], d["preds_neutralized"]))


print(f"Spearman score for high feature exposure: {spearman_unbalanced_scores_per_era.mean()}")
print(f"Spearman score for balanced feature expo: {spearman_balanced_scores_per_era.mean()}")

print(f"std for high feature exposure: {spearman_unbalanced_scores_per_era.std(ddof=0)}")
print(f"std for balanced feature expo: {spearman_balanced_scores_per_era.std(ddof=0)}")

try:
  print(f"smart sharpe for high feature exposure: {smart_sharpe(spearman_unbalanced_scores_per_era)}")
  print(f"smart sharpe for balanced feature expo: {smart_sharpe(spearman_balanced_scores_per_era)}")
except ZeroDivisionError:
  print("Division by zero!!!")
  pass

In [ ]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = df2["preds_neutralized"]
predictions_df.head()

In [ ]:
predictions_df["prediction_kazutsugi"].describe()

## 5. Make your first submission
To enter the tournament, we must submit the predictions back to Numerai. We will use the `numerapi` library to do this.

In [ ]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "BX5I7ZCKH3HK2U337GQSTKIPQWEBPLTQ"
secret_key = "DJK3BNVUPQNY2IAWCEHEPLEGFEXD2K6U6SOHA67F7U4U2MA7KJXED3XRAAEU4SMP"
model_id = "37686618-db61-44d0-ade0-30d176beedd0"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
#Upload your predictions
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)